# TP 3 - Partie 2 : Transfer Learning en Vision 🖼️

Dans ce notebook, nous allons appliquer le transfer learning à la vision par ordinateur.

**Objectifs :**
1. Charger un modèle de vision pré-entraîné léger
2. Comprendre la différence entre Feature Extraction et Fine-tuning
3. Entraîner sur un nouveau dataset
4. Visualiser les prédictions

⚠️ **Contrainte matérielle** : Nous utilisons des modèles légers (ResNet18 ~11M params) adaptés aux PCs de faculté.

## 1. Setup et imports

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Subset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

from tqdm.notebook import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device : {device}")

# Fixer les seeds pour la reproductibilité
torch.manual_seed(42)
np.random.seed(42)

## 2. Chargement du dataset

Nous utilisons **CIFAR-10** (10 classes, images 32×32) avec un sous-ensemble pour l'entraînement rapide.

Les classes : avion, voiture, oiseau, chat, cerf, chien, grenouille, cheval, bateau, camion

In [ ]:
# Classes CIFAR-10
CLASSES = ['avion', 'voiture', 'oiseau', 'chat', 'cerf', 
           'chien', 'grenouille', 'cheval', 'bateau', 'camion']

# Transformations pour le train (data augmentation)
train_transform = transforms.Compose([
    transforms.Resize(224),  # ResNet attend 224×224
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Transformations pour le test
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Télécharger CIFAR-10
print("Téléchargement de CIFAR-10...")
full_train = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform
)
full_test = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform
)

# Réduire la taille pour l'entraînement rapide
# Prendre seulement 1000 images par classe pour l'entraînement
train_indices = []
for class_idx in range(10):
    class_indices = [i for i, (_, label) in enumerate(full_train) if label == class_idx]
    train_indices.extend(class_indices[:1000])  # 1000 par classe

train_dataset = Subset(full_train, train_indices)
test_dataset = full_test  # Garder tout le test

print(f"\nDataset réduit :")
print(f"   Entraînement : {len(train_dataset)} images")
print(f"   Test : {len(test_dataset)} images")

In [ ]:
# Visualiser quelques exemples
def denormalize(tensor):
    """Enlever la normalisation pour l'affichage"""
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    # Trouver une image de la classe i
    for img, label in train_dataset:
        if label == i:
            img = denormalize(img)
            ax.imshow(img.permute(1, 2, 0).clamp(0, 1))
            ax.set_title(CLASSES[i])
            ax.axis('off')
            break
plt.tight_layout()
plt.show()

### DataLoaders

In [ ]:
# Créer les DataLoaders
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

print(f"Batches d'entraînement : {len(train_loader)}")
print(f"Batches de test : {len(test_loader)}")

# Vérifier la shape d'un batch
images, labels = next(iter(train_loader))
print(f"\nShape d'un batch d'images : {images.shape}")
print(f"   → [batch_size={images.shape[0]}, channels={images.shape[1]}, H={images.shape[2]}, W={images.shape[3]}]")

## 3. Chargement d'un modèle pré-entraîné

Nous utilisons **ResNet18**, un modèle léger (~11M paramètres) pré-entraîné sur ImageNet.

**Architecture de ResNet18 :**
- 4 blocs résiduels (layers)
- Skip connections pour éviter le vanishing gradient
- ~11M paramètres (rapide à entraîner)

In [ ]:
# Charger ResNet18 pré-entraîné
resnet = models.resnet18(pretrained=True)

print("=== Architecture ResNet18 ===")
print(resnet)

# Compter les paramètres
total = sum(p.numel() for p in resnet.parameters())
print(f"\nTotal paramètres : {total:,} (~{total/1e6:.1f}M)")

In [ ]:
# Explorer la structure
print("=== Structure hiérarchique ===")
for name, module in resnet.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"{name:15s} : {module.__class__.__name__:20s} ({params:,} params)")

print("\n=== La dernière couche (classifier) ===")
print(f"fc : {resnet.fc}")
print(f"\nCette couche a été entraînée pour classifier sur 1000 classes ImageNet.")
print(f"Nous allons la remplacer pour 10 classes CIFAR-10.")

## 4. Stratégie 1 : Feature Extraction (Geler le backbone)

**Principe :** On garde les poids du modèle pré-entraîné figés et on n'entraîne que la dernière couche (classifier).

**Avantages :**
- Très rapide (moins de paramètres à entraîner)
- Peu de données nécessaires
- Évite le overfitting

**Cas d'usage :** Petit dataset, ressources limitées

In [ ]:
# Créer le modèle pour feature extraction
model_fe = models.resnet18(pretrained=True)

# Geler tous les paramètres du backbone
for param in model_fe.parameters():
    param.requires_grad = False

# Remplacer la dernière couche pour 10 classes
num_features = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_features, 10)

# Seuls les paramètres de la nouvelle couche sont entraînables
trainable = sum(p.numel() for p in model_fe.parameters() if p.requires_grad)
total = sum(p.numel() for p in model_fe.parameters())

print(f"Feature Extraction :")
print(f"   Paramètres entraînables : {trainable:,}")
print(f"   Paramètres figés : {total - trainable:,}")
print(f"   Taux d'entraînement : {trainable/total*100:.2f}%")

model_fe = model_fe.to(device)

In [ ]:
# Fonction d'entraînement
def train_model(model, train_loader, test_loader, epochs=5, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    train_losses, test_losses = [], []
    train_accs, test_accs = [], []
    
    for epoch in range(epochs):
        # Entraînement
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Évaluation
        model.eval()
        test_loss, test_acc = evaluate(model, test_loader, criterion)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")
    
    return train_losses, test_losses, train_accs, test_accs

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100. * correct / total

# Entraîner (3 epochs pour aller vite)
print("\n=== Entraînement Feature Extraction ===")
history_fe = train_model(model_fe, train_loader, test_loader, epochs=3, lr=0.001)

## 5. Stratégie 2 : Fine-tuning complet

**Principe :** On dégèle tout le modèle et on entraîne avec un learning rate faible.

**Avantages :**
- Meilleures performances
- Adaptation complète au nouveau domaine

**Inconvénients :**
- Plus lent
- Risque de overfitting
- Nécessite plus de données

**Astuce :** Utiliser un learning rate plus faible pour les couches pré-entraînées.

In [ ]:
# Créer le modèle pour fine-tuning
model_ft = models.resnet18(pretrained=True)

# Remplacer le classifier (tous les paramètres sont entraînables par défaut)
num_features = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_features, 10)
model_ft = model_ft.to(device)

trainable = sum(p.numel() for p in model_ft.parameters() if p.requires_grad)
print(f"Fine-tuning : {trainable:,} paramètres entraînables")

# Entraîner avec un LR plus faible
print("\n=== Entraînement Fine-tuning ===")
history_ft = train_model(model_ft, train_loader, test_loader, epochs=3, lr=0.0001)

## 6. Comparaison des deux stratégies

In [ ]:
# Visualiser la comparaison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
axes[0].plot(history_fe[2], label='Feature Extraction', marker='o')
axes[0].plot(history_ft[2], label='Fine-tuning', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Accuracy sur le train')
axes[0].legend()
axes[0].grid(True)

# Test accuracy
axes[1].plot(history_fe[3], label='Feature Extraction', marker='o')
axes[1].plot(history_ft[3], label='Fine-tuning', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Accuracy sur le test')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\n=== Résultats finaux ===")
print(f"Feature Extraction - Test Acc : {history_fe[3][-1]:.2f}%")
print(f"Fine-tuning        - Test Acc : {history_ft[3][-1]:.2f}%")

## 7. Visualisation des prédictions

In [ ]:
# Visualiser quelques prédictions
def visualize_predictions(model, dataset, num_images=10):
    model.eval()
    
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    
    indices = np.random.choice(len(dataset), num_images, replace=False)
    
    for idx, ax in zip(indices, axes.flat):
        image, label = dataset[idx]
        
        # Prédiction
        with torch.no_grad():
            output = model(image.unsqueeze(0).to(device))
            pred = output.argmax(1).item()
            probs = torch.softmax(output, dim=1)[0]
        
        # Afficher
        img_display = denormalize(image).permute(1, 2, 0).clamp(0, 1)
        ax.imshow(img_display)
        color = 'green' if pred == label else 'red'
        ax.set_title(f"Vrai: {CLASSES[label]}\nPréd: {CLASSES[pred]}\nConf: {probs[pred]:.1%}", 
                    color=color)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

print("Prédictions du modèle Fine-tuned :")
visualize_predictions(model_ft, test_dataset)

### Matrice de confusion

In [ ]:
# Calculer la matrice de confusion
def get_predictions(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(1).cpu()
            
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_preds), np.array(all_labels)

preds, labels = get_predictions(model_ft, test_loader)

# Afficher
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel('Prédiction')
plt.ylabel('Vérité')
plt.title('Matrice de confusion')
plt.show()

# Rapport de classification
print("\nRapport de classification :")
print(classification_report(labels, preds, target_names=CLASSES, digits=3))

## 8. Sauvegarde et chargement du modèle

In [ ]:
# Sauvegarder le modèle
save_path = "./mon_modele_cifar10.pth"
torch.save({
    'model_state_dict': model_ft.state_dict(),
    'classes': CLASSES,
    'model_name': 'resnet18'
}, save_path)

print(f"Modèle sauvegardé : {save_path}")

# Charger le modèle
checkpoint = torch.load(save_path)

# Recréer l'architecture
loaded_model = models.resnet18(pretrained=False)
loaded_model.fc = nn.Linear(loaded_model.fc.in_features, 10)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model = loaded_model.to(device)

print("Modèle chargé avec succès !")

# Vérifier qu'il fonctionne
_, acc = evaluate(loaded_model, test_loader, nn.CrossEntropyLoss())
print(f"Accuracy du modèle chargé : {acc:.2f}%")

## 🎯 Récapitulatif

Dans ce notebook, nous avons vu :

1. **Chargement de modèle pré-entraîné** : ResNet18 depuis `torchvision.models`
2. **Feature Extraction** : Geler le backbone, entraîner seulement le classifier (rapide)
3. **Fine-tuning** : Entraîner tout le modèle avec un LR faible (meilleures perfs)
4. **Évaluation** : Visualisation des prédictions et matrice de confusion
5. **Sauvegarde** : `torch.save()` et `torch.load()`

**Règles de pouce pour choisir :**
- **Petit dataset (<1000 images)** → Feature Extraction
- **Dataset moyen (1000-10000)** → Fine-tuning avec LR faible
- **Gros dataset (>10000)** → Fine-tuning ou entraînement from scratch

## ✏️ Exercices optionnels

1. **Essayer un autre modèle** : Remplacer ResNet18 par `mobilenet_v2` (encore plus léger)
2. **Data augmentation** : Ajouter plus d'augmentations et observer l'impact
3. **Learning rate scheduling** : Utiliser `torch.optim.lr_scheduler` pour réduire le LR
4. **Early stopping** : Arrêter l'entraînement quand la val accuracy stagne
5. **Grad-CAM** : Visualiser quelles parties de l'image le modèle regarde